In [1]:
#%matplotlib inline 
import torch 
#from IPython import display 
#from matplotlib import pyplot as plt
import numpy as np 
import random
import math
#from time import time
#from torch import optim

In [2]:
def generatedata(noise,showpermutation=False,showtrue_w=False):
    true_w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if showtrue_w:
        print('true_w2:',true_w2)
    X2_before_ =torch.from_numpy(np.random.normal(0, 1, (num_example, 1)))
    y_ = torch.exp(true_w2[0]*X2_before_)*torch.sin(true_w2[1]*X2_before_)
    y_ += torch.from_numpy(np.random.normal(0, noise ,size=y_.size()))
    P_array=np.random.permutation(num_example)
    P=torch.zeros(num_example,num_example,dtype=torch.float64)
    for i in range(num_example):
        P[i][P_array[i]]=1
    if showpermutation:
        print('打乱X2的置换矩阵为',P)
    X2_=torch.mm(P,X2_before_)
    conditionnumber=np.linalg.cond(X2_.numpy())
    #X2_=X2_before_
    error_reg=(torch.norm(y_-torch.exp(true_w2[0]*X2_before_)*torch.sin(true_w2[1]*X2_before_))/torch.norm(y_))
    return y_,X2_,true_w2,P,error_reg,conditionnumber

In [3]:
def generateinitialw(method,showinitialw=False):
    if method=='normal':
        w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if method=='zeros':
        w2=torch.zeros(num_X2feature,1,dtype=torch.float64)
    if showinitialw:
        print('initial w2:',w2)
    return w2

In [4]:
def sinkhorn_stabilized(a, b, M, reg, numItermax=1000, tau=1e3, stopThr=1e-9,
                        warmstart=None, verbose=False, print_period=20,
                        log=False, **kwargs):

#     a = np.asarray(a, dtype=np.float64)
#     b = np.asarray(b, dtype=np.float64)
#     M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M

#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # test if multiple target
#     if len(b.shape) > 1:
#         n_hists = b.shape[1]
#         a = a[:, np.newaxis]
#     else:
#         n_hists = 0
    n_hists = 0
    # init data
    dim_a = len(a)
    dim_b = len(b)

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    if n_hists:
        u = torch.ones((dim_a, n_hists)) / dim_a
        v = torch.ones((dim_b, n_hists)) / dim_b
    else:
        u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))- beta.reshape((1, dim_b))) / reg)

    def get_Gamma(alpha, beta, u, v):
        """log space gamma computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1)) - beta.reshape((1, dim_b)))
                      / reg + torch.log(u.reshape((dim_a, 1))) + torch.log(v.reshape((1, dim_b))))

    # print(np.min(K))

    K = get_K(alpha, beta)
    transp = K
    loop = 1
    cpt = 0
    err = 1
    while loop:

        uprev = u
        vprev = v
        # sinkhornrn update
        v = b / (torch.mm(K.transpose(1,0), u) + 1e-16)
        u = a / (torch.mm(K, v) + 1e-16)
        # remove numerical problems and store them in K
        if torch.abs(u).max() > tau or torch.abs(v).max() > tau:
            if n_hists:
                alpha, beta = alpha + reg * \
                    torch.max(torch.log(u), 1), beta + reg * torch.max(np.log(v))
            else:
                alpha, beta = alpha + reg * torch.log(u), beta + reg * torch.log(v)
                if n_hists:
                    u, v = torch.ones((dim_a, n_hists)) / dim_a, torch.ones((dim_b, n_hists)) / dim_b
                else:
                    u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b
            K = get_K(alpha, beta)
            

        if cpt % print_period == 0:
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            if n_hists:
                err_u = abs(u - uprev).max()
                err_u /= max(abs(u).max(), abs(uprev).max(), 1.)
                err_v = abs(v - vprev).max()
                err_v /= max(abs(v).max(), abs(vprev).max(), 1.)
                err = 0.5 * (err_u + err_v)
            else:
                transp = get_Gamma(alpha, beta, u, v)
                err = torch.norm((torch.sum(transp, axis=0) - b))
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 20) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr:
            loop = False

        if cpt >= numItermax:
            loop = False

        if np.any(np.isnan(u.detach().numpy())) or np.any(np.isnan(v.detach().numpy())):
            # we have reached the machine precision
            # come back to previous solution and quit loop
            print('Warning: numerical errors at iteration', cpt)
            u = uprev
            v = vprev
            break

        cpt = cpt + 1
    #print(cpt)
    if log:
        if n_hists:
            alpha = alpha[:, None]
            beta = beta[:, None]
        logu = alpha / reg + torch.log(u)
        logv = beta / reg + torch.log(v)
        log['logu'] = logu
        log['logv'] = logv
        log['alpha'] = alpha + reg * torch.log(u)
        log['beta'] = beta + reg * torch.log(v)
        log['warmstart'] = (log['alpha'], log['beta'])
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res, log

        else:
            return get_Gamma(alpha, beta, u, v), log
    else:
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res
        else:
            return get_Gamma(alpha, beta, u, v)

In [5]:
def sinkhorn_epsilon_scaling(a, b, M, reg, numItermax=100, epsilon0=1e4,
                             numInnerItermax=100, tau=1e3, stopThr=1e-9,
                             warmstart=None, verbose=False, print_period=10,
                             log=False, **kwargs):
    #a = np.asarray(a, dtype=np.float64)
    #b = np.asarray(b, dtype=np.float64)
    #M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M
#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # init data
    dim_a = len(a)
    #dim_a=num_example
    dim_b = len(b)
    #dim_b=num_example
    # nrelative umerical precision with 64 bits
    numItermin = 35
    numItermax = max(numItermin, numItermax)  # ensure that last velue is exact

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))
                        - beta.reshape((1, dim_b))) / reg)

    # print(np.min(K))
    def get_reg(n):  # exponential decreasing
        return (epsilon0 - reg) * np.exp(-n) + reg

    loop = 1
    cpt = 0
    err = 1
    while loop:

        regi = get_reg(cpt)

        G, logi = sinkhorn_stabilized(a, b, M, regi,
                                      numItermax=numInnerItermax, stopThr=1e-9,
                                      warmstart=(alpha, beta), verbose=False,
                                      print_period=20, tau=tau, log=True)

        alpha = logi['alpha']
        beta = logi['beta']

        if cpt >= numItermax:
            loop = False

        if cpt % (print_period) == 0:  # spsion nearly converged
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            transp = G
            err = torch.norm(
                (torch.sum(transp, axis=0) - b))**2 + torch.norm((torch.sum(transp, axis=1) - a))**2
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 10) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr and cpt > numItermin:
            loop = False

        cpt = cpt + 1
    # print('err=',err,' cpt=',cpt)
    if log:
        log['alpha'] = alpha
        log['beta'] = beta
        log['warmstart'] = (log['alpha'], log['beta'])
        return G, log
    else:
        return G
    

In [13]:
gama_=1
eta=0
starts=1
for i_____ in range(5):
    for num_example in range(100,101,100): 
        for num_X2feature in [2]:
            for i____ in range(10):
                (y_,X2_,true_w2,true_P,error_reg1,conditionnumber)=generatedata(noise=0.1)
                y=y_
                X2=X2_
                results_Loss = []
                results_w2=[]
                results_error=[]
                for i__ in range(starts):
    #                     P_array=np.random.permutation(num_example)
    #                     P=torch.zeros(num_example,num_example,dtype=torch.float64)
    #                     for i in range(num_example):
    #                         P[i][P_array[i]]=1
    #                     X_=torch.cat([X1,X2],1)
    #                     X=torch.mm(P,X_)
    #                     w=torch.mm(torch.mm(torch.tensor(np.linalg.inv(torch.mm(X.transpose(1,0),X))),X.transpose(1,0)),y)
    #                     w1,w2=w.split([num_X1feature,num_X2feature],dim=0)
    #                     w1=torch.from_numpy(np.random.normal(0, 0,(num_X1feature,1)))
    #                     w2=torch.from_numpy(np.random.normal(0, 0,(num_X2feature,1)))
                    w2=generateinitialw(method='zeros')
                    #w2=true_w2
                    w2.requires_grad_(requires_grad=True)
    #                 results_Loss = []
                    lr=0.002
                    results_S=[]
                    t=0
                    before1=0
                    while True:                     
                        Y1=y
                        Y2=torch.exp(w2[0]*X2)*torch.sin(w2[1]*X2)
                        C=torch.zeros(num_example,num_example,dtype=torch.float64)
                        for i in range(num_example):
                            for j in range(num_example):
                                C[i][j]=(Y1[i]-Y2[j])**2            

                        #S=SinkhornIPOT(C)
                        a=torch.ones(num_example,1,dtype=torch.float64)
                        b=torch.ones(num_example,1,dtype=torch.float64)
                        S=sinkhorn_epsilon_scaling(a, b, C, 0.00000001)
                        #print(S.transpose(1,0).half())
                        #results_S.append(S)
                        #if t>0:
                            #print('        S变化',(torch.norm(results_S[t]-results_S[t-1]))/(torch.norm(results_S[t-1])))
                        #Loss=torch.sum(S*C)
                        Loss=torch.norm(Y1-torch.mm(S,Y2))**2
                        if Loss<1e-2:
                            break
                        Loss.backward()
    #                         results_Loss.append(Loss)
    #                         for i_ in range(num_X1features):
    #                             results_w1[t][i_]=(w1[i_].data)
    #                         for i_ in range(num_X2features):
    #                             results_w2[t][i_]=(w2[i_].data)
                        w2.data-=lr*(w2.grad+np.random.normal(0,np.sqrt(eta/(1+t)**gama_)))
                        #print(w2.grad)
    #                     if t==num_epochs-1:
    #                         print('最终w1梯度：',w1.grad)
    #                         print('最终w2梯度：',w2.grad)
                        w2.grad.data.zero_() 
                        print('Loss',t,'={:.4f}'.format(Loss.data))
                        #print('Loss',t,'=',Loss)
    #                     if t%6==0:
    #                         if torch.norm(Loss-before1)<1e-4:
    #                             break
    #                         before1=Loss
                        if torch.norm(Loss-before1)/before1<1e-5:
                            break
                        before1=Loss
                        if t>=150:
                            print('超过迭代上限')
                            break
                        if math.isnan(Loss):
                            break
                        t+=1
                        print('#',end='')



                    print(' ',end='')
                    error_each=(torch.norm(w2-true_w2))
                    #results_error.append(error_each)
                    error_each2=(torch.norm(w2-true_w2))/torch.norm(true_w2)
                    
                    #results_Loss.append(Loss)
                    #results_w1.append(w1.data)
                    #results_w2.append(w2.data)


                #w1=results_w1[results_Loss.index(min(results_Loss))]
                #w2=results_w2[results_Loss.index(min(results_Loss))]

    #                     for i_ in range(starts):
    #                         results_w1[i_]=(w1[i_].data)
    #                     for i_ in range(starts):
    #                         results_w2[i_]=(w2[i_].data)


                #error_w=((torch.norm(w1-true_w1))/(torch.norm(true_w1))+(torch.norm(w2-true_w2))/(torch.norm(true_w2)))/2
                #print(num_X1feature,num_X2feature,num_example,'平均相对误差1：',error_w)
                #print(num_X2feature,num_example,'平均相对误差2：',np.min(results_error),end='   ')
                print(num_X2feature,num_example,'绝对误差',error_each,end='   ')
                print('相对误差',error_each2,end='   ')
                #print('真实置换矩阵为：',true_P)    
                
                error_reg2=(torch.norm(y_-torch.exp(w2[0]*torch.mm(S,X2_))*torch.sin(w2[1]*torch.mm(S,X2_)))/torch.norm(y_))
                print('真实回归误差',error_reg1,end='   ')
                print('实验回归误差',error_reg2,end='   ')
                error_P=(torch.norm(S.transpose(1,0)-true_P))/(torch.norm(true_P))
                print('置换矩阵误差',error_P,end='   ')
                matrix_count=0
                for i_2 in range(num_example):
                    for j_2 in range(num_example):
                        if (abs(S[i_2][j_2]-1)<0.02):
                            matrix_count+=1
                print('离置换矩阵距离',matrix_count,end='   ')
                print('条件数',conditionnumber)
                #print('双随机矩阵S为：',S.transpose(1,0).half())
                #print(results)
    #                 plt.figure(figsize=(6,6))
    #                 plt.plot(results_w1_0,results_Loss, '-o',label='$w1[0]$')
    #                 plt.plot(results_w1_1,results_Loss, '-o',label='$w1[1]$')
    #                 plt.plot(results_w1_2,results_Loss, '-o',label='$w1[2]$')
    #                 plt.plot(results_w2_0,results_Loss, '-o',label='$w2[0]$')
    #                 plt.plot(results_w2_1,results_Loss, '-o',label='$w2[1]$')
    #                 plt.plot(results_w2_2,results_Loss, '-o',label='$w2[2]$')
    #                 plt.legend()
    #                 plt.show()

Loss 0 =50.8659
#Loss 1 =50.7801
#Loss 2 =21.9877
#Loss 3 =12.2403
#Loss 4 =7.1721
#Loss 5 =4.2150
#Loss 6 =2.6507
#Loss 7 =1.9020
#Loss 8 =1.5380
#Loss 9 =1.3268
#Loss 10 =1.1802
#Loss 11 =1.0674
#Loss 12 =0.9773
#Loss 13 =0.9049
#Loss 14 =0.8464
#Loss 15 =0.7988
#Loss 16 =0.7599
#Loss 17 =0.7224
#Loss 18 =0.6910
#Loss 19 =0.6652
#Loss 20 =0.6425
#Loss 21 =0.6215
#Loss 22 =0.6024
#Loss 23 =0.5845
#Loss 24 =0.5676
#Loss 25 =0.5519
#Loss 26 =0.5352
#Loss 27 =0.5207
#Loss 28 =0.5068
#Loss 29 =0.4956
#Loss 30 =0.4864
#Loss 31 =0.4783
#Loss 32 =0.4714
#Loss 33 =0.4653
#Loss 34 =0.4604
#Loss 35 =0.4563
#Loss 36 =0.4527
#Loss 37 =0.4497
#Loss 38 =0.4467
#Loss 39 =0.4442
#Loss 40 =0.4421
#Loss 41 =0.4403
#Loss 42 =0.4388
#Loss 43 =0.4376
#Loss 44 =0.4367
#Loss 45 =0.4359
#Loss 46 =0.4350
#Loss 47 =0.4339
#Loss 48 =0.4330
#Loss 49 =0.4322
#Loss 50 =0.4315
#Loss 51 =0.4309
#Loss 52 =0.4304
#Loss 53 =0.4300
#Loss 54 =0.4296
#Loss 55 =0.4294
#Loss 56 =0.4291
#Loss 57 =0.4290
#Loss 58 =0.4288
#Los

#Loss 14 =5.0806
#Loss 15 =4.9026
#Loss 16 =4.7479
#Loss 17 =4.6126
#Loss 18 =4.4939
#Loss 19 =4.3891
#Loss 20 =4.2962
#Loss 21 =4.2133
#Loss 22 =4.1392
#Loss 23 =4.0726
#Loss 24 =4.0126
#Loss 25 =3.9583
#Loss 26 =3.9089
#Loss 27 =3.8640
#Loss 28 =3.8230
#Loss 29 =3.7854
#Loss 30 =3.7510
#Loss 31 =3.7193
#Loss 32 =3.6902
#Loss 33 =3.6634
#Loss 34 =3.6385
#Loss 35 =3.6155
#Loss 36 =3.5941
#Loss 37 =3.5742
#Loss 38 =3.5557
#Loss 39 =3.5384
#Loss 40 =3.5223
#Loss 41 =3.5072
#Loss 42 =3.4930
#Loss 43 =3.4797
#Loss 44 =3.4673
#Loss 45 =3.4556
#Loss 46 =3.4446
#Loss 47 =3.4342
#Loss 48 =3.4245
#Loss 77 =3.7705
#Loss 78 =3.7372
#Loss 79 =3.7067
#Loss 80 =3.6786
#Loss 81 =3.6526
#Loss 82 =3.6285
#Loss 83 =3.6062
#Loss 84 =3.5855
#Loss 85 =3.5661
#Loss 86 =3.5482
#Loss 87 =3.5314
#Loss 88 =3.5158
#Loss 89 =3.5011
#Loss 90 =3.4873
#Loss 91 =3.4744
#Loss 92 =3.4623
#Loss 93 =3.4509
#Loss 94 =3.4403
#Loss 95 =3.4304
#Loss 96 =3.4213
#Loss 97 =3.4130
#Loss 98 =3.4061
#Loss 99 =3.4013
#Loss 100 =3.4

#Loss 92 =51.7515
#Loss 93 =44.6275
#Loss 94 =37.7841
#Loss 95 =27.7154
#Loss 96 =9.1473
#Loss 97 =4.6074
#Loss 98 =2.9181
#Loss 99 =3.4650
#Loss 100 =24.9689
#Loss 101 =101.0124
#Loss 102 =431.3847
#Loss 103 =15135469.0664
#Loss 104 =nan
 2 100 绝对误差 tensor(nan, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(nan, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1022, dtype=torch.float64)   实验回归误差 tensor(nan, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 tensor(1.0000, dtype=torch.float64, grad_fn=<DivBackward0>)   离置换矩阵距离 0   条件数 1.0
Loss 0 =6.8961
#Loss 1 =6.8846
#Loss 2 =2.6235
#Loss 3 =1.0801
#Loss 4 =0.5007
#Loss 5 =0.2702
#Loss 6 =0.1738
#Loss 7 =0.1318
#Loss 8 =0.1126
#Loss 9 =0.1035
#Loss 10 =0.0989
#Loss 11 =0.0963
#Loss 12 =0.0948
#Loss 13 =0.0937
#Loss 14 =0.0929
#Loss 15 =0.0923
#Loss 16 =0.0918
#Loss 17 =0.0914
#Loss 18 =0.0910
#Loss 19 =0.0907
#Loss 20 =0.0905
#Loss 21 =0.0902
#Loss 22 =0.0900
#Loss 23 =0.0898
#Loss 24 =0.0897
#Loss 25 =0

#Loss 32 =0.1222
#Loss 33 =0.1214
#Loss 34 =0.1207
#Loss 35 =0.1199
#Loss 36 =0.1193
#Loss 37 =0.1186
#Loss 38 =0.1180
#Loss 39 =0.1174
#Loss 40 =0.1168
#Loss 41 =0.1162
#Loss 42 =0.1157
#Loss 43 =0.1152
#Loss 44 =0.1147
#Loss 45 =0.1143
#Loss 46 =0.1138
#Loss 47 =0.1134
#Loss 48 =0.1130
#Loss 49 =0.1126
#Loss 50 =0.1122
#Loss 51 =0.1118
#Loss 52 =0.1115
#Loss 53 =0.1111
#Loss 54 =0.1108
#Loss 55 =0.1105
#Loss 56 =0.1102
#Loss 57 =0.1099
#Loss 58 =0.1097
#Loss 59 =0.1094
#Loss 60 =0.1092
#Loss 61 =0.1089
#Loss 62 =0.1087
#Loss 63 =0.1085
#Loss 64 =0.1083
#Loss 65 =0.1080
#Loss 66 =0.1079
#Loss 67 =0.1077
#Loss 68 =0.1075
#Loss 69 =0.1073
#Loss 70 =0.1071
#Loss 71 =0.1070
#Loss 72 =0.1068
#Loss 73 =0.1067
#Loss 74 =0.1065
#Loss 75 =0.1064
#Loss 76 =0.1063
#Loss 77 =0.1061
#Loss 78 =0.1060
#Loss 79 =0.1059
#Loss 80 =0.1058
#Loss 81 =0.1057
#Loss 82 =0.1056
#Loss 83 =0.1055
#Loss 84 =0.1054
#Loss 85 =0.1053
#Loss 86 =0.1052
#Loss 87 =0.1051
#Loss 88 =0.1050
#Loss 89 =0.1050
#Loss 90 =0.10

#Loss 6 =2.1203
#Loss 7 =1.6287
#Loss 8 =1.3159
#Loss 9 =1.1071
#Loss 10 =0.9629
#Loss 11 =0.8592
#Loss 12 =0.7799
#Loss 13 =0.7180
#Loss 14 =0.6714
#Loss 15 =0.6332
#Loss 16 =0.5976
#Loss 17 =0.5710
#Loss 18 =0.5510
#Loss 19 =0.5359
#Loss 20 =0.5244
#Loss 21 =0.5155
#Loss 22 =0.5082
#Loss 23 =0.5025
#Loss 24 =0.4981
#Loss 25 =0.4944
#Loss 26 =0.4916
#Loss 27 =0.4894
#Loss 28 =0.4877
#Loss 29 =0.4864
#Loss 30 =0.4854
#Loss 31 =0.4846
#Loss 32 =0.4840
#Loss 33 =0.4836
#Loss 34 =0.4832
#Loss 35 =0.4829
#Loss 36 =0.4823
#Loss 37 =0.4818
#Loss 38 =0.4814
#Loss 39 =0.4811
#Loss 40 =0.4809
#Loss 41 =0.4807
#Loss 42 =0.4805
#Loss 43 =0.4804
#Loss 44 =0.4803
#Loss 45 =0.4802
#Loss 46 =0.4797
#Loss 47 =0.4793
#Loss 48 =0.4789
#Loss 49 =0.4780
#Loss 50 =0.4773
#Loss 51 =0.4768
#Loss 52 =0.4764
#Loss 53 =0.4761
#Loss 54 =0.4759
#Loss 55 =0.4757
#Loss 56 =0.4755
#Loss 57 =0.4754
#Loss 58 =0.4753
#Loss 59 =0.4752
#Loss 60 =0.4751
#Loss 61 =0.4750
#Loss 62 =0.4749
#Loss 63 =0.4748
#Loss 64 =0.4748
#

#Loss 42 =0.4176
#Loss 43 =0.4158
#Loss 44 =0.4143
#Loss 45 =0.4131
#Loss 46 =0.4121
#Loss 47 =0.4114
#Loss 48 =0.4107
#Loss 49 =0.4102
#Loss 50 =0.4098
#Loss 51 =0.4095
#Loss 52 =0.4093
#Loss 53 =0.4091
#Loss 54 =0.4089
#Loss 55 =0.4088
#Loss 56 =0.4087
#Loss 57 =0.4086
#Loss 58 =0.4085
#Loss 59 =0.4084
#Loss 60 =0.4084
#Loss 61 =0.4084
#Loss 62 =0.4083
#Loss 63 =0.4083
#Loss 64 =0.4083
#Loss 65 =0.4083
#Loss 66 =0.4083
#Loss 67 =0.4083
#Loss 68 =0.4083
#Loss 69 =0.4082
#Loss 70 =0.4082
#Loss 71 =0.4082
#Loss 72 =0.4082
#Loss 73 =0.4082
#Loss 74 =0.4082
 2 100 绝对误差 tensor(1.8020, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(1.9605, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1842, dtype=torch.float64)   实验回归误差 tensor(0.1039, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 tensor(1.4000, dtype=torch.float64, grad_fn=<DivBackward0>)   离置换矩阵距离 100   条件数 1.0
Loss 0 =146.1893
#Loss 1 =145.5979
#Loss 2 =93.3722
#Loss 3 =61.4042
#Loss 4 =23.9826
#Loss 

#Loss 8 =3.8394
#Loss 9 =3.4467
#Loss 10 =3.1573
#Loss 11 =2.9335
#Loss 12 =2.7516
#Loss 13 =2.6149
#Loss 14 =2.5023
#Loss 15 =2.4006
#Loss 16 =2.3084
#Loss 17 =2.2205
#Loss 18 =2.1358
#Loss 19 =2.0557
#Loss 20 =1.9780
#Loss 21 =1.9074
#Loss 22 =1.8368
#Loss 23 =1.7796
#Loss 24 =1.7308
#Loss 25 =1.6744
#Loss 26 =1.6246
#Loss 27 =1.5848
#Loss 28 =1.5493
#Loss 29 =1.5155
#Loss 30 =1.4848
#Loss 31 =1.4571
#Loss 32 =1.4345
#Loss 33 =1.4138
#Loss 34 =1.3959
#Loss 35 =1.3804
#Loss 36 =1.3671
#Loss 37 =1.3555
#Loss 38 =1.3465
#Loss 39 =1.3396
#Loss 40 =1.3334
#Loss 41 =1.3270
#Loss 42 =1.3215
#Loss 43 =1.3158
#Loss 44 =1.3116
#Loss 45 =1.3082
#Loss 46 =1.3052
#Loss 47 =1.3028
#Loss 48 =1.3010
#Loss 49 =1.2998
#Loss 50 =1.2988
#Loss 51 =1.2980
#Loss 52 =1.2973
#Loss 53 =1.2969
#Loss 54 =1.2965
#Loss 55 =1.2962
#Loss 56 =1.2958
#Loss 57 =1.2952
#Loss 58 =1.2948
#Loss 59 =1.2944
#Loss 60 =1.2939
#Loss 61 =1.2936
#Loss 62 =1.2933
#Loss 63 =1.2930
#Loss 64 =1.2927
#Loss 65 =1.2925
#Loss 66 =1.2924

 2 100 绝对误差 tensor(0.0128, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(0.0217, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1610, dtype=torch.float64)   实验回归误差 tensor(0.0701, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 tensor(1.3115, dtype=torch.float64, grad_fn=<DivBackward0>)   离置换矩阵距离 100   条件数 1.0
Loss 0 =58.7963
#Loss 1 =57.5065
#Loss 2 =29.6470
#Loss 3 =19.2145
#Loss 4 =13.2063
#Loss 5 =9.7574
#Loss 6 =8.2832
#Loss 7 =7.9012
#Loss 8 =7.8479
#Loss 9 =7.8433
#Loss 10 =7.8429
#Loss 11 =7.8429
 2 100 绝对误差 tensor(1.6747, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(1.6703, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.1412, dtype=torch.float64)   实验回归误差 tensor(0.3652, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 tensor(1.4142, dtype=torch.float64, grad_fn=<DivBackward0>)   离置换矩阵距离 100   条件数 1.0
Loss 0 =11392.9969
#Loss 1 =11361.5830
#Loss 2 =10880.3486
#Loss 3 =10454.8588
#Loss 4 =11849.2924
#Loss 5 =

#Loss 134 =91.3752
#Loss 135 =147.7498
#Loss 136 =96.8960
#Loss 137 =83.3245
#Loss 138 =124.8594
#Loss 139 =70.2635
#Loss 140 =72.6675
#Loss 141 =85.8624
#Loss 142 =96.5158
#Loss 143 =157.3553
#Loss 144 =108.0938
#Loss 145 =78.0601
#Loss 146 =100.4597
#Loss 147 =82.9545
#Loss 148 =119.3459
#Loss 149 =69.9092
#Loss 150 =69.5581
超过迭代上限
 2 100 绝对误差 tensor(4.0461, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(2.7195, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0580, dtype=torch.float64)   实验回归误差 tensor(0.4657, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 tensor(1.4000, dtype=torch.float64, grad_fn=<DivBackward0>)   离置换矩阵距离 100   条件数 1.0
Loss 0 =124.2547
#Loss 1 =122.6196
#Loss 2 =91.1249
#Loss 3 =72.1351
#Loss 4 =48.8895
#Loss 5 =27.3938
#Loss 6 =19.0448
#Loss 7 =15.2552
#Loss 8 =13.1399
#Loss 9 =16.7147
#Loss 10 =51.2098
#Loss 11 =115.5624
#Loss 12 =103.3947
#Loss 13 =92.7816
#Loss 14 =76.0374
#Loss 15 =51.6238
#Loss 16 =24.6851
#Loss 17 =14.0243


#Loss 85 =0.0783
#Loss 86 =0.0783
#Loss 87 =0.0783
#Loss 88 =0.0783
#Loss 89 =0.0783
#Loss 90 =0.0783
#Loss 91 =0.0783
#Loss 92 =0.0783
#Loss 93 =0.0783
#Loss 94 =0.0783
#Loss 95 =0.0783
#Loss 96 =0.0783
#Loss 97 =0.0783
#Loss 98 =0.0783
#Loss 99 =0.0783
#Loss 100 =0.0783
#Loss 101 =0.0783
#Loss 102 =0.0783
#Loss 103 =0.0783
#Loss 104 =0.0783
#Loss 105 =0.0783
#Loss 106 =0.0783
#Loss 107 =0.0783
#Loss 108 =0.0783
 2 100 绝对误差 tensor(0.4479, dtype=torch.float64, grad_fn=<NormBackward0>)   相对误差 tensor(1.5086, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.5424, dtype=torch.float64)   实验回归误差 tensor(0.1552, dtype=torch.float64, grad_fn=<DivBackward0>)   置换矩阵误差 tensor(1.4070, dtype=torch.float64, grad_fn=<DivBackward0>)   离置换矩阵距离 100   条件数 1.0
Loss 0 =14.5087
#Loss 1 =13.9934
#Loss 2 =6.8718
#Loss 3 =4.6590
#Loss 4 =3.6191
#Loss 5 =2.9582
#Loss 6 =2.4860
#Loss 7 =2.1363
#Loss 8 =1.8718
#Loss 9 =1.6664
#Loss 10 =1.5024
#Loss 11 =1.3683
#Loss 12 =1.2566
#Loss 13 =1.1624
#Loss 1